In [186]:
import sys
sys.path.append('/Users/Jessica/Desktop/Coding/School/2802ICT/Assignment2')
from imports import *

In [189]:

class NN:

    def __init__(self, n_input = 784, n_hidden = 30, n_out = 10, epoch = 30, mini_b = 20, l_rate = 3):
        self.feature_train = []
        self.feature_test = []
        self.target_train = []
        self.target_test = []

        self.N_input = n_input
        self.N_hidden = n_hidden
        self.N_out = n_out
        
        self.epoch = epoch 
        self.minibatch = mini_b
        self.l_rate = l_rate

        self.weights = self.get_weights()


    def arrange_data(self, train_file, test_file):
        
        train_set = (self.open_data(train_file)) #/255.astype('float32')
        test_set = (self.open_data(test_file)) #/255.astype('float32')

        self.target_train, self.feature_train = self.split_data(train_set)
        self.target_test, self.feature_test = self.split_data(test_set)

    def split_data(self, data):
        #seperates a data set into the targets and features
        label_set = data["label"]
        df = pd.DataFrame(data)
        feature_set = df.drop('label', axis=1)

        return label_set, feature_set

    def open_data(self, file):
        data_set = pd.read_csv(file)
        return data_set

    def get_weights(self):

        parameters = {
            'W1': np.random.randn(self.N_hidden, self.N_input) * np.sqrt(1. / self.N_hidden),
            'W2': np.random.randn(self.N_out, self.N_hidden) * np.sqrt(1. / self.N_out)
            }
        return parameters

    def forward(self, feature_train):
        
        weights = self.weights
        weights['A0'] = feature_train

        #input layer to hidden layer
        weights['Z1'] = np.dot(weights['weight1'], weights['A0'])
        weights['A1'] = self.sigmoid(weights['Z1'])

        #hidden layer to the output layer
        weights['Z2'] = np.dot(weights['weights2'], weights['A1'])
        weights['A2'] = self.softmax(weights['Z2'])

        return weights['A2']

    def backprop(self, feature, output):
        weights = self.weights
        new_weights = []

        error = 2 * (output - feature) / output.shape[0] * self.softmax(weights['Z2'], dev = True)
        new_weights['W2'] = np.outer(error, weights['A1'])

        error = np.dot(weights['W2'].T, error) * self.sigmoid(weights['Z1'], dev = True)
        new_weights['W1'] = np.outer(error, weights['A0'])

        return new_weights

    def sigmoid(self, z, derivative = False):
        if derivative:
            return (np.exp(-z))/((np.exp(-z)+1)**2)
        return 1.0 / (1.0 + np.exp(-z))

    def softmax(self, z, deriv = False):
       
        exps = np.exp(z - z.max())
        if deriv:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis = 0))
        
        return exps / np.sum(exps, axis = 0)

    def train_net(self, feature, target):
        
        for i in range(self.epoch):
            for 

    def driver(self, train_file, test_file):
        self.arrange_data(train_file, test_file)



SyntaxError: invalid syntax (<ipython-input-189-6c287191e723>, line 71)

In [188]:

####################### USER SETUP AREA ####################### 

n_input = 784
n_hidden = 30
n_out = 10
epoch = 30
mini_b = 20
l_rate = 3
    
####################### END OF USER SETUP #######################

nn = NN(n_input, n_hidden, n_out, epoch, mini_b, l_rate)
nn.driver("fashion-mnist_train.csv.gz", "fashion-mnist_test.csv.gz")


       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       5       0   
3           0       0       0       1       2       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
59995       0       0       0       0       0       0       0       0       0   
59996       0       0       0       0       0       0       0       0       0   
59997       0       0       0       0       0       0       0       0       0   
59998       0       0       0       0       0       0       0       0       0   
59999       0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel7